<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2020ccap/2020-0404neural_networks_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
- source https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py
- date: 2020-0404
- note: for AI_arts
---


In [0]:
%matplotlib inline


# Neural Networks


ニューラルネットワークは `` torch.nn``パッケージを使用して構築できます。
`autograd` を概説しました。`nn` モジュールは `autograd` に依存して，モデルを定義し，勾配計算を行います。
`nn.Module` には層と `output` を返すメソッド `forward(input)` が含まれます。
数字画像を分類する LeNet を見てください:

<div align="center">
    <img src="https://pytorch.org/tutorials/_images/mnist.png"><br/>
    2020ccap/mnist.png
</div>

<!--
It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``
-->

これは単純なフィードフォワードネットワークです。 入力を受け取り，いくつかの層を逐次通過し，最終的に出力を提供します。
ニューラルネットワークの一般的なトレーニング手順は次のとおりです。

- 学習可能なパラメーター（または重み）を持つニューラルネットワークを定義する
- 入力のデータセットを反復処理する
- ネットワークを通過させて入力情報を処理する
- 損失を計算する（出力が正しいことからどれくらい離れているか）
- 勾配をネットワークのパラメータに伝播する
- 単純な更新則を使用してネットワークの結合係数を更新する: 
   ``結合係数 = 以前の結合係数 - 学習率 * 勾配``

## ネットワークの定義
ネットワークを定義してみましょう

<!--
## Define the network
Let’s define this network:
-->


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 以下で実際にネットワークを定義します
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 入力画像のチャンネル数は 1 すなわち白黒濃淡画像
        # 出力チャンネルすなわち特徴数は 6 ,
        # 3 x 3 の畳み込みを実施
        self.conv1 = nn.Conv2d(1, 6, 3)  # 左から順に 入力次元数，出力次元数，カーネルサイズ
        self.conv2 = nn.Conv2d(6, 16, 3)
        #  アフィン変換: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 縦横 6*6 の画像が 16 次元の特徴（カーネル）数から 120 次元の出力
        self.fc2 = nn.Linear(120, 84)          # 入力次元 120 から 出力 84 次元へ変換
        self.fc3 = nn.Linear(84, 10)           # 入力 84 次元から 10 次元へ 

    def forward(self, x):
        # ウィンドウ幅 (2, 2) のマックスプーリング
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 正方行列でウィンドウを指定するのであれば引数は 1 つでも良い
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 最初の次元はミニバッチサイズなので除外
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


<!--
You just have to define the ``forward`` function, and the ``backward``
function (where gradients are computed) is automatically defined for you using ``autograd``.
You can use any of the Tensor operations in the ``forward`` function.

The learnable parameters of a model are returned by ``net.parameters()``
-->

単に `forward` 関数を定義するだけで `autograd` 機能により，勾配が計算される `backward` 関数は自動的に定義されます。
`forward` 関数では テンソル Tensor 操作を使用できます。

モデルの学習可能なパラメータは `net.parameters()` によって返されます。

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [0]:
#print(params)

In [6]:
# keras スタイルのネットワーク要約を出力するパッケージ torchsummary を使います。
#!pip install torchsummary
import torchsummary
torchsummary.summary(net, input_size=(1,32,32))  # input_size の引数は 入力チャンネル数（特徴数）, x (幅), y（高さ）

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 30, 30]              60
            Conv2d-2           [-1, 16, 13, 13]             880
            Linear-3                  [-1, 120]          69,240
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 81,194
Trainable params: 81,194
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.31
Estimated Total Size (MB): 0.38
----------------------------------------------------------------


<!--Let's try a random 32x32 input.
Note: expected input size of this net (LeNet) is 32x32. 
To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.
-->

ランダムな 32x32 の入力を試してみましょう。
注：このネット（LeNet）の想定入力サイズは 32x32 です。
MNIST データセットでこのネットを使用するにはデータセットから 32x32 に画像のサイズを変更してください。


In [7]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0691, -0.0019,  0.1468,  0.1439,  0.0606, -0.0514,  0.1081, -0.0990,
         -0.1274,  0.0289]], grad_fn=<AddmmBackward>)


<!--Zero the gradient buffers of all parameters and backprops with random gradients:-->

勾配計算に必要な全パラメータを 0 で初期化し，誤差逆伝播用の勾配を乱数で初期化します。


In [0]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<!--
<div class="alert alert-info">
    <h4>Note</h4><p>
    
``torch.nn`` only supports mini-batches. The entire ``torch.nn`` package only supports inputs that are a mini-batch of samples, and not a single sample.

    For example, ``nn.Conv2d`` will take in a 4D Tensor of ``nSamples x nChannels x Height x Width``.

    If you have a single sample, just use ``input.unsqueeze(0)`` to add a fake batch dimension.
</p>
</div>
-->

<div class="aleart-info">
<h3>注: </h3><p>
`torch.nn` はミニバッチのみをサポートします。`torch.nn` パッケージ全体は，単一のサンプルではなく，サンプルのミニバッチである入力のみをサポートします。
たとえば `nn.Conv2d` は `nSamples x nChannels x Height x Width` の 4元テンソルを取り込みます。
単一のサンプルがある場合は `input.unsqueeze(0)` を使用して偽のミニバッチ次元を追加します。
</div>

**これまでの復習**:
<!--
Before proceeding further, let's recap all the classes you’ve seen so far.

**Recap:**
-->

-  ``torch.Tensor`` - 多次元配列。自動微分 ``backward()`` に対応。その勾配を保持している<!--A *multi-dimensional array* with support for autograd operations like ``backward()``. Also *holds the gradient* w.r.t. the tensor.-->
-  ``nn.Module`` - ニューラルネットワークモジュール。パラメータの格納，GPU への移動，読み込み，書き出しなどをサポートしている<!--
Neural network module. *Convenient way of encapsulating parameters*, with helpers for moving them to GPU, exporting, loading, etc.-->
-  ``nn.Parameter`` - PyTorch で用いられるテンソルの一つ。各パラメータの属性や値は PyTorhc で用いられる <!--A kind of Tensor, that is *automatically registered as a parameter when assigned as an attribute to a* ``Module``.-->
-  ``autograd.Function`` - ``forward（）`` を定義すれば，自動的に ``backward()`` を計算してくれる。各々の ``Function`` ノードは，``Tensor`` を生成し，その履歴を保持している <--Implements *forward and backward definitions of an autograd operation*. Every ``Tensor`` operation creates at least a single ``Function`` node that connects to functions that created a ``Tensor`` and *encodes its history*.-->


**ここまでの説明**:
- ニューラルネットワークの定義
- 入力と `backword` 演算の処理

<!--**At this point, we covered:**
  -  Defining a neural network
  -  Processing inputs and calling backward
-->

**この先**:
- 損失関数の定義
- パラメータの更新(学習)


<!--
**Still Left:**
  -  Computing the loss
  -  Updating the weights of the network
-->


## 損失関数
<!--
## Loss Function
-->

- <font color="teal">損失関数</font>は，出力とターゲット（正解）の対を入力して，出力がターゲットとどのくらい離れているかを計算します。
- PyTorch では損失関数として，あらかじめ，数種類定義されています。 詳細は <https://pytorch.org/docs/nn.html#loss-functions> を御覧ください。
もっとも簡単な平均自乗誤差関数は ``nn.MSELoss`` です。

<!--
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different `loss functions` <https://pytorch.org/docs/nn.html#loss-functions> under the
nn package.
A simple loss is: ``nn.MSELoss`` which computes the mean-squared error between the input and the target.

For example:
-->

In [0]:
#help(torch.nn)
#help(torch.nn.functional)

In [10]:
output = net(input)
target = torch.randn(10)  # ダミーの値を定義
target = target.view(1, -1)  # 出力と同じ次元にする
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5948, grad_fn=<MseLossBackward>)


In [11]:
# 少し分かりづらいかも知れないので解説，基本単位が必ずミニバッチサイズなので，Tensor の次元は 1 次元ではありえない
# ですので，次元を揃えるために target.view(1,-1)  をします。
output.size()

torch.Size([1, 10])

損失関数 ``loss`` を逆伝播させるために ``.grad_fn`` を使います。計算グラフとしては以下のようになります。
<!--
Now, if you follow ``loss`` in the backward direction, using its ``.grad_fn`` attribute, you will see a graph of computations that lookslike this:
-->

```

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss
```

``loss.backword()`` を呼び出すと，あらかじめ定義してあった ``損失`` 値に基づいて，計算グラフ全体が微分されます。
``requires_grad=True`` と設定されている，計算グラフ中の全テンソルが計算されます。``.grad`` テンソルには累積勾配が保持されます。

<!--
So, when we call ``loss.backward()``, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True`` will have their ``.grad`` Tensor accumulated with the gradient.
-->

実習:
<!--
For illustration, let us follow a few steps backward:
-->


In [12]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# 誤差逆伝播
<!--
# Backprop

To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients though, else gradients will be
accumulated to existing gradients.
-->


``loss.backward()`` を呼び出して，`conv1` のバイアスを確認しまししょう。
<!--
Now we shall call ``loss.backward()``, and have a look at conv1's bias gradients before and after the backward.
-->


In [13]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0085, -0.0063,  0.0046,  0.0052, -0.0058, -0.0040])


## 損失関数の用い方

ほとんどのニューラルネットワークフレームワークでは あらかじめ頻用される損失関数 loss functions が定義されています。
詳細は [https://pytorch.org/docs/nn](https://pytorch.org/docs/nn) を御覧ください。

**The only thing left to learn is:**

## 結合係数の更新 （学習，あるいは 訓練）

確率的勾配降下法 (SGD) の簡単な場合は以下のようになります:

     ``weight = weight - learning_rate * gradient``

Python のコードでは以下のようになります

```python
    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
```

それ以外の更新方法，しばしば最適化手法と呼ばれます。
PyTorch では ``torch.optim`` で定義されています。
<!--
However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. 
To enable this, we built a small package: ``torch.optim`` that implements all these methods. 
Using it is very simple:
-->

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

<!--
Observe how gradient buffers had to be manually set to zero using  ``optimizer.zero_grad()``. 
This is because gradients are accumulated  as explained in the `Backprop`section.
-->